In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import preprocessing, model_selection, metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
train = pd.read_csv("train.csv", parse_dates=["activation_date"])
test = pd.read_csv("test.csv", parse_dates=["activation_date"])

In [189]:
train_df = train[:10000]
test_df = test[:5000]

In [7]:
print("Train file rows and columns are : ", train_df.shape)
print("Test file rows and columns are : ", test_df.shape)

Train file rows and columns are :  (10000, 18)
Test file rows and columns are :  (5000, 17)


In [8]:
train_y = train_df["deal_probability"].values
test_id = test_df["item_id"].values

In [10]:
train_df["activation_weekday"] = train_df["activation_date"].dt.weekday
test_df["activation_weekday"] = test_df["activation_date"].dt.weekday
train_df["Weekd of Year"] = train_df['activation_date'].dt.week
test_df["Weekd of Year"] = test_df['activation_date'].dt.week
train_df["Day of Month"] = train_df['activation_date'].dt.day
test_df["Day of Month"] = test_df['activation_date'].dt.day

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [11]:
train_df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,activation_weekday,Weekd of Year,Day of Month
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),...,400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789,1,13,28
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,...,3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000,6,12,26
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,...,4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177,0,12,20
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,...,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323,5,12,25
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",...,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797,3,11,16


In [12]:
cat_vars = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3"]

In [13]:
for col in cat_vars:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [15]:
textfeats = ["description", "title"]
for cols in textfeats:
    train_df[cols] = train_df[cols].astype(str) 
    train_df[cols] = train_df[cols].astype(str).fillna('nicapotato') # FILL NA
    train_df[cols] = train_df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    train_df[cols + '_num_chars'] = train_df[cols].apply(len) # Count number of Characters
    train_df[cols + '_num_words'] = train_df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    train_df[cols + '_num_unique_words'] = train_df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    train_df[cols + '_words_vs_unique'] = train_df[cols+'_num_unique_words'] / train_df[cols+'_num_words'] * 100 # Count Unique Words
    test_df[cols] = test_df[cols].astype(str) 
    test_df[cols] = test_df[cols].astype(str).fillna('nicapotato') # FILL NA
    test_df[cols] = test_df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    test_df[cols + '_num_chars'] = test_df[cols].apply(len) # Count number of Characters
    test_df[cols + '_num_words'] = test_df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    test_df[cols + '_num_unique_words'] = test_df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    test_df[cols + '_words_vs_unique'] = test_df[cols+'_num_unique_words'] / test_df[cols+'_num_words'] * 100 # Count Unique Words

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [16]:
train_df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,...,Weekd of Year,Day of Month,description_num_chars,description_num_words,description_num_unique_words,description_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
0,b912c3c6a6ad,e00f8ff2eaf9,19,182,4,41,181,41,274,кокоби(кокон для сна),...,13,28,58,7,7,100.0,21,3,3,100.0
1,2dac0150717d,39aeb48f0017,17,511,2,22,86,41,274,стойка для одежды,...,12,26,41,7,7,100.0,17,3,3,100.0
2,ba83aefab5dc,91e2f88dd6e3,16,502,0,2,52,41,274,philips bluray,...,12,20,99,17,17,100.0,14,2,2,100.0
3,02996f1dd2ea,bf5cccea572d,21,378,4,41,32,41,274,автокресло,...,12,25,22,3,3,100.0,10,1,1,100.0
4,7c90be56d2ab,ef50846afc0b,4,123,6,0,204,52,21,"ваз 2110, 2003",...,11,16,24,4,4,100.0,14,3,3,100.0


In [17]:
train_df["price"].fillna(train_df.price.mean(),inplace=True)
train_df["image_top_1"].fillna(-999,inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [18]:
cols_to_drop = ["item_id", "user_id", "activation_date", "image", "title", "description"]
train_X = train_df.drop(cols_to_drop + ["deal_probability"], axis=1)

In [19]:
test_X = test_df.drop(cols_to_drop, axis=1)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(
        train_X, train_y, test_size=0.20, random_state=23)

In [37]:
params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 30,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }

In [22]:
lgtrain = lgb.Dataset(X_train, label=y_train)

In [23]:
lgval = lgb.Dataset(X_valid, label=y_valid)

In [25]:
evals_result = {}
model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=20, evals_result=evals_result)

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's rmse: 0.238338
[40]	valid_0's rmse: 0.23673
[60]	valid_0's rmse: 0.236897
[80]	valid_0's rmse: 0.23729
[100]	valid_0's rmse: 0.238277
[120]	valid_0's rmse: 0.238507
Early stopping, best iteration is:
[39]	valid_0's rmse: 0.236718


In [37]:
pred_test = model.predict(test_X, num_iteration=model.best_iteration)

In [38]:
pred_test[pred_test>1] = 1
pred_test[pred_test<0] = 0
sub_df = pd.DataFrame({"item_id":test_id})
sub_df["deal_probability"] = pred_test
sub_df.to_csv("baseline_lgb.csv", index=False)

### Linear Regression

In [26]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
lr = linear_model.LinearRegression(normalize=True)

In [27]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [28]:
predicted_lr = lr.predict(X_valid)

In [29]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
rng = np.random.RandomState(1)

In [30]:
ab = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=1000, random_state=rng)

In [31]:
ab.fit(X_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=1000,
         random_state=<mtrand.RandomState object at 0x0000008D0ED0C9D8>)

In [32]:
predicted_ab = ab.predict(X_valid)

In [33]:
predicted_en = (predicted_lr+predicted_ab)/2

In [36]:
print("Mean squared error: %.2f"
      % np.sqrt(mean_squared_error(y_valid, predicted_en)))

Mean squared error: 0.25
